In [1]:
pip install panda_gym torchvision av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 58.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 16.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import gymnasium as gym
import panda_gym

from stable_baselines3 import PPO
from stable_baselines3.common.policies import ActorCriticPolicy
from stable_baselines3.common.callbacks import BaseCallback

import torch
import torch.nn as nn
import torchvision.models as models

import panda_ppo_utils
from panda_ppo_utils import (
                             ObservationWrapper, 
                             FeaturesExtractor,
                             ValidationCallback,
)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
resnet18 = models.resnet18(pretrained=True)
feature_extractor = torch.nn.Sequential(*list(resnet18.children())[:-1]) 
feature_extractor.eval()  

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 171MB/s] 


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [6]:
train_env = gym.make("PandaPickAndPlace-v3", render_mode="rgb_array")
train_env = ObservationWrapper(train_env)

val_env = gym.make("PandaPickAndPlace-v3", render_mode="rgb_array")
val_env = ObservationWrapper(val_env)

policy_kwargs = dict(
    features_extractor_class=FeaturesExtractor,
    features_extractor_kwargs=dict(feature_extractor=feature_extractor, features_dim=258),  
    net_arch=dict(pi=[128], vf=[128])
)

model = PPO(
    policy="CnnPolicy",
    env=train_env,
    policy_kwargs=policy_kwargs,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    learning_rate=3e-4,
    gamma=0.99,
    gae_lambda=0.95,
    verbose=1,
    device=device
)

validation_callback = ValidationCallback(
    validation_env=val_env,
    validation_steps=20000, 
    output_filename="validation_video",
    fps=10,
    max_steps=200,
    verbose=1
)

pybullet build time: Nov 28 2023 23:45:17


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [14]:
model.learn(total_timesteps=1000000, callback=validation_callback)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 50       |
|    ep_rew_mean     | -50      |
| time/              |          |
|    fps             | 4        |
|    iterations      | 1        |
|    time_elapsed    | 491      |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 49.4         |
|    ep_rew_mean          | -49.4        |
| time/                   |              |
|    fps                  | 3            |
|    iterations           | 2            |
|    time_elapsed         | 1040         |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0061873477 |
|    clip_fraction        | 0.0669       |
|    clip_range           | 0.2          |
|    e

KeyboardInterrupt: 